## Import packages

In [1]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

## Import Data and DataLoader

In [3]:
transforms_func = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
train_datasets = datasets.ImageFolder(
    'PetImages/train',
    transform=transforms_func
)
val_datasets = datasets.ImageFolder(
    'PetImages/val',
    transform=transforms_func
)
test_datasets = datasets.ImageFolder(
    'PetImages/test',
    transform=transforms_func
)

In [4]:
batch_size = 16
train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_datasets, batch_size=batch_size, shuffle=False)

## Model Building

In [5]:
class AlexNet(nn.Module):
    def __init__(self, ) -> None:
        super().__init__()
        self.features_extractor = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Flatten()
        )

        # self.fc1 = nn.Linear(9216, 4096)
        # self.fc2 = nn.Linear(4096, 4096)
        # self.fc3 = nn.Linear(4096, 1)
        self.fcn = nn.Sequential(
            nn.Dropout(),
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, 4096),
            nn.ReLU(inplace=1),

            nn.Linear(4096, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x.shape == 3, 227, 227
        x = self.features_extractor(x)
        x = self.fcn(x)
        return x

## Training Pipeline / Loop

In [6]:
def train(model, 
          criteria, 
          optimizer, 
          train_dataloader, 
          val_dataloader, 
          epochs, 
          device, 
          val_iteration=1,
          ):
    train_loss = []
    val_loss = []
    val_accuracy = []

    for epoch in range(1, epochs+1):
        ## Train 
        model.train()
        epoch_train_loss = 0
        train_step = 0
        for batch_images, batch_labels in tqdm(train_dataloader):
            batch_images = batch_images.to(device)
            batch_labels = batch_labels.to(device)

            optimizer.zero_grad()
            # forward pass
            results = model(batch_images)
            loss = criteria(results, batch_labels.unsqueeze(1).float())

            # backward pass
            loss.backward() # calcuate gradients
            optimizer.step() # update weights using gradients
            train_step += 1
            epoch_train_loss += loss.item()
        epoch_train_loss /= train_step
        train_loss.append(epoch_train_loss)
        print(f"Epoch {epoch} of {epochs}: train loss = {epoch_train_loss:.4f}")

        ## Validation 
        if epoch % val_iteration == 0:
            model.eval()

            epoch_val_loss = 0
            val_step = 0

            epoch_val_correct = 0
            with torch.no_grad():
                for batch_images, batch_labels in val_dataloader:
                    batch_images = batch_images.to(device)
                    batch_labels = batch_labels.to(device)
                    
                    # Validation only have forward pass no backward pass 
                    # forward pass 
                    

                    results = model(batch_images)
                    loss = criteria(results, batch_labels.unsqueeze(1).float())
                    epoch_val_loss += loss.item()
                    val_step += 1
                    epoch_val_correct += (results.round() == batch_labels.unsqueeze(1)).type(torch.float).sum().item()

            epoch_val_loss /= val_step
            val_loss.append(epoch_val_loss)
            epoch_val_accuracy = epoch_val_correct / len(val_dataloader.dataset)
            val_accuracy.append(epoch_val_accuracy)
            print(f"Epoch {epoch} of {epochs}: validation loss = {epoch_val_loss:.4f}")
            print(f"Epoch {epoch} of {epochs}: validation Accu = {epoch_val_accuracy:.4f}")
    return train_loss, val_loss, val_accuracy


## Hyper parameter and Training

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
epochs = 10 

alexnet = AlexNet().to(device)
criteria = nn.BCELoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.001)

In [9]:
train_loss, val_loss, val_accuracy = train(alexnet, criteria, optimizer, train_dataloader, val_dataloader, epochs, device)

 19%|█▉        | 210/1094 [00:46<03:13,  4.56it/s]


KeyboardInterrupt: 